# Build an AI Agent to analyze e-commerce patterns

In [ ]:
!pip install -U "ibm-watsonx-ai" | tail -n 1
!pip install -U "pydantic" | tail -n 1
!pip install -U "langchain" | tail -n 1
!pip install -U "langchain-ibm" | tail -n 1
!pip install -U "langchain-community" | tail -n 1
!pip install -U "langchain_huggingface" | tail -n 1
!pip install -U "sqlalchemy" | tail -n 1
!pip install -U "pymilvus" | tail -n 1
!pip install -U "PyMuPDF" | tail -n 1

In [1]:
import pandas as pd
import sqlalchemy
from langchain.sql_database import SQLDatabase
import contextlib
from tempfile import TemporaryFile
from tabulate import tabulate

@contextlib.contextmanager
def get_ecommerce_db():
    with TemporaryFile(suffix=".db") as f:
        # Read CSV
        sales_data = pd.read_csv('data/sales_data.csv')
        product_inventory = pd.read_csv('data/product_inventory.csv')
        customer_purchase_history = pd.read_csv('data/customer_purchase_history.csv')

        # Create SQL Engine
        engine = sqlalchemy.create_engine(f"sqlite:///{f.name}")
    
        # Load data into SQL tables
        sales_data.to_sql("sales_data", con=engine, index=False, if_exists="replace")
        product_inventory.to_sql("product_inventory", con=engine, index=False, if_exists="replace")
        customer_purchase_history.to_sql("customer_purchase_history", con=engine, index=False, if_exists="replace")
    
        yield SQLDatabase.from_uri(f"sqlite:///{f.name}")
        engine.dispose(close=True)

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm.autonotebook import tqdm, trange
from pymilvus import MilvusClient
from langchain_huggingface import HuggingFaceEmbeddings

def rag_load(collection_name) -> MilvusClient:
    loader = TextLoader('data/reviews.txt')
    docs = loader.load()
    print("Number of pages:", len(docs))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, is_separator_regex = False)
    splits = text_splitter.split_documents(docs)
    print("Number of splits:", len(splits))
    milvus_client = MilvusClient("./milvus_rag.db")
    if milvus_client.has_collection(collection_name):
        milvus_client.drop_collection(collection_name)
    milvus_client.create_collection(
        collection_name=collection_name,
        dimension=768,
        metric_type="IP",  # Inner product distance
        consistency_level="Strong",  # Strong consistency level
    )
    data = []
    for i, line in enumerate(tqdm(splits, desc="Creating embeddings please wait...")):
        data.append({"id": i, "vector": embeddings.embed_query(line.page_content), "text": line.page_content})
    milvus_client.insert(collection_name=collection_name, data=data)
    return milvus_client

def rag_retriever(query: str) -> str:
    result = ""
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=[
            embeddings.embed_query(query)
        ],
        limit=2,
        search_params={"metric_type": "IP", "params": {}},
        output_fields=["text"],
    )
    retrieved_lines_with_distances = [(res["entity"]["text"], res["distance"]) for res in search_res[0]]
    for item in retrieved_lines_with_distances:
        result += "[Document]\n"
        result += item[0] + "\n"
        result += "[End]\n\n"
    return result.strip()

embeddings = HuggingFaceEmbeddings()
collection_name = 'my_rag_collection'
milvus_client = rag_load(collection_name)

Number of pages: 1
Number of splits: 4


Creating embeddings please wait...:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import os

os.environ["WX_URL"] = "https://us-south.ml.cloud.ibm.com"
os.environ["WX_PROJECT_ID"] = "xxxx-xxxx-xxxx-xxxx"
os.environ["WX_API_KEY"] = "<your-ibmcloud-api-key>"

In [5]:
import os
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
from langchain_ibm import WatsonxLLM

def initialize_watsonx_llm():
    parameters = {
        GenTextParamsMetaNames.MAX_NEW_TOKENS: 250,
        GenTextParamsMetaNames.MIN_NEW_TOKENS: 20,
        GenTextParamsMetaNames.DECODING_METHOD: 'greedy',
    }
    llm = WatsonxLLM(
        url=os.environ["WX_URL"],
        apikey=os.environ["WX_PROJECT_ID"],
        project_id=os.environ["WX_API_KEY"],
        model_id="mistralai/mistral-large",
        params=parameters
    )
    response = llm.invoke('What is a generative AI? Respond concisely.')
    print(response)
    return llm

llm = initialize_watsonx_llm()



Generative AI is a type of artificial intelligence that can create new content, such as images, music, or text. It learns patterns in data and uses that learning to generate new, likely examples. Examples include DALL-E for images and ChatGPT for text.


In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder

In [7]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation.
Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools only if necessary.
Reminder to ALWAYS list the available tables before executing any SQL query on a table.
Reminder to ALWAYS check for relationship between tables if column name is not found.
"""

user_prompt = """{input}
{agent_scratchpad}
(reminder to respond in a JSON blob no matter what)"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_prompt
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", user_prompt),
    ]
)

In [8]:
from langchain.agents import tool
from langchain.tools import Tool

@tool("milvus_retriever")
def milvus_retriever(query: str) -> str:
    """
    Returns the matching occurance of the query from the vectorstore.

    Args:
        query: The query string to match in the vectorstore.

    Returns:
        str: Nearest occurances of the query string.
    """
    formatted_response = rag_retriever(query)
    
    return formatted_response

rag_tool = Tool(
    name="milvus_retriever",
    func=milvus_retriever,
    description="Search for information in the vector database.",
)

In [9]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

with get_ecommerce_db() as db:
    sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    tools_chat = sql_toolkit.get_tools()

tools_chat.append(rag_tool) # This will be all the tools that you need for your AI Agent

In [10]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain_core.agents import AgentAction, AgentFinish

class CustomJSONAgentOutputParser(JSONAgentOutputParser):
    def parse(self, text: str) -> AgentAction | AgentFinish:
        i = text.find("{")

        # Sometimes model may return additional tokens before json blob
        text = text[i:]
        if "```" not in text:
            text = "\n```\n" + text
        return super().parse(text + "\n```\nObservation:")

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str

prompt_chat = prompt.partial(
    tools=render_text_description(list(tools_chat)),
    tool_names=", ".join([t.name for t in tools_chat]),
)

# Setting stop sequence for LLM
llm_with_stop = llm.bind(stop=["}\n"])

agent_chat = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt_chat
    | llm_with_stop
    | CustomJSONAgentOutputParser()
)

agent_executor_chat = AgentExecutor(
    agent=agent_chat, tools=tools_chat, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True
)

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor_chat,
    get_session_history=lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [13]:
import time 

def agentExecutorWithExecTime(input_text: str, session_id: str) -> dict:
    start_time = time.time()
    answer = agent_with_chat_history.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}},
    )
    end_time = time.time()
    execution_time = end_time - start_time
    return {
        "input": answer['input'],
        "llm_reasoning": answer['intermediate_steps'],
        "chat_history": answer['chat_history'],
        "output": answer['output'],
        "execution_time": f'{round(execution_time, 2)} sec'
    }

In [14]:
answer1 = agentExecutorWithExecTime("Which product did Alice buy?", "session1")



> Entering new AgentExecutor chain...

```
{
  "action": "sql_db_list_tables",
  "action_input": ""
}

```
Observation:customer_purchase_history, product_inventory, sales_data
```
{
  "action": "sql_db_schema",
  "action_input": "customer_purchase_history, product_inventory, sales_data"
}

```
Observation:
CREATE TABLE customer_purchase_history (
	"CustomerID" BIGINT, 
	"CustomerName" TEXT, 
	"ProductID" TEXT, 
	"PurchaseDate" TEXT, 
	"Quantity" BIGINT
)

/*
3 rows from customer_purchase_history table:
CustomerID	CustomerName	ProductID	PurchaseDate	Quantity
101	Alice	P001	2023-06-01	2
102	Bob	P002	2023-06-02	1
103	Charlie	P003	2023-06-03	1
*/


CREATE TABLE product_inventory (
	"ProductID" TEXT, 
	"ProductName" TEXT, 
	"Category" TEXT, 
	"Stock" BIGINT, 
	"Price" FLOAT
)

/*
3 rows from product_inventory table:
ProductID	ProductName	Category	Stock	Price
P001	iPhone	Phones	50	999.99
P002	iPad	Tablets	100	329.99
P003	Mac	Computers	30	1199.99
*/


CREATE TABLE sales_data (
	"OrderID" BI

In [15]:
print(answer1['output'])

Alice bought iPhone and AirPods.


In [16]:
print(answer1['execution_time'])

9.67 sec


In [17]:
answer2 = agentExecutorWithExecTime("What was her reaction to those products?", "session1")



> Entering new AgentExecutor chain...

```
{
  "action": "milvus_retriever",
  "action_input": "Alice's reaction to iPhone and AirPods"
}

```
Observation:[Document]
MentionID: M003
Text: "Just got the new Mac, and it's amazing! #apple #computers"
ProductID: P003
Platform: Instagram
Name: Charlie

MentionID: M004
Text: "The AirPods have great sound quality. #apple #accessories"
ProductID: P004
Platform: Instagram
Name: Alice

MentionID: M005
Text: "The new iPhone is impressive. Can't wait to explore all its features. #apple"
ProductID: P001
Platform: Twitter
Name: David

MentionID: M006
Text: "iPad is my go-to device for reading and browsing. #apple #ipad"
ProductID: P002
Platform: Facebook
Name: Frank

MentionID: M007
Text: "My new Mac is a powerhouse. Perfect for my design work. #apple #mac"
ProductID: P003
Platform: Instagram
Name: Charlie

MentionID: M008
Text: "The AirPods' noise cancellation is top-notch. #apple #airpods"
ProductID: P004
Platform: Twitter
Name: Alice

MentionID

In [18]:
print(answer2['output'])

Alice had a positive reaction to the iPhone, mentioning that it is fantastic with great build quality and very useful. However, she had a negative reaction to the AirPods, stating that they are not as good as she expected and broke after a week.


In [19]:
print(answer2['execution_time'])

3.69 sec
